In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt

import time
import os
import cv2
from PIL import Image

from glob import glob
from tqdm import tqdm

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

C:\Users\jeffg\AppData\Local\Temp\ipykernel_47624\1379983671.py:19: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 用 UTKface_inthewild

```
Labels
The labels of each face image is embedded in the file name, formated like [age]_[gender]_[race]_[date&time].jpg

[age] is an integer from 0 to 116, indicating the age
[gender] is either 0 (male) or 1 (female)
[race] is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).
[date&time] is in the format of yyyymmddHHMMSSFFF, showing the date and time an image was collected to UTKFace

```

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
raw_face_img_list = pd.read_pickle( r"C:\Users\jeffg\face-project\face-api\jupyter\face_img_list.pkl")
raw_face_label_list = pd.read_pickle(r"C:\Users\jeffg\face-project\face-api\jupyter\face_label_list.pkl")

In [4]:
face_img_list = []
face_label_list = []
for idx, f_name in enumerate(raw_face_label_list):
    if len(f_name.split("_")) != 4:
        print(f_name)
    else:
        face_label_list.append(raw_face_label_list[idx])
        face_img_list.append(raw_face_img_list[idx])

61_1_20170109142408075.jpg
61_3_20170109150557335.jpg
39_1_20170116174525125.jpg


In [5]:
X_train, X_test, y_train, y_test = train_test_split(face_img_list, face_label_list, test_size=0.33, random_state=42)

In [7]:
for img_array, f_name in tqdm(zip(X_train, y_train)):
    age_, gender_ = f_name.replace("__", "_").split("_")[:2]
    
    age = "%d-%d"%((int(age_)//5)*5 , (int(age_)//5 + 1)*5)
    gender = ["male", "female"][int(gender_)]

    img = Image.fromarray(img_array)

    ## age
    os.makedirs(os.path.join("UTKface_age", "train", age) , exist_ok=True)
    img.save(os.path.join("UTKface_age", "train", age , f_name))

    ## gender
    os.makedirs(os.path.join("UTKface_gender", "train", gender) , exist_ok=True)
    img.save(os.path.join("UTKface_gender", "train", gender , f_name))
    

for img_array, f_name in tqdm(zip(X_test, y_test)):
    age_, gender_ = f_name.replace("__", "_").split("_")[:2]
    
    age = "%d-%d"%((int(age_)//5)*5 , (int(age_)//5 + 1)*5)
    gender = ["male", "female"][int(gender_)]

    img = Image.fromarray(img_array)

    ## age
    os.makedirs(os.path.join("UTKface_age", "test", age) , exist_ok=True)
    img.save(os.path.join("UTKface_age", "test", age , f_name))

    ## gender
    os.makedirs(os.path.join("UTKface_gender", "test", gender) , exist_ok=True)
    img.save(os.path.join("UTKface_gender", "test", gender , f_name))

16108it [00:50, 319.04it/s]
7934it [00:24, 322.89it/s]


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO(r"C:\Users\jeffg\face-project\age_gender\Face-Gender-Classification-PyTorch\runs\classify\UTKface_Gender\weights\best.pt")  # load a pretrained model (recommended for training)

# Train the model
# results = model.train(data="UTKface_Age", epochs=100, imgsz=640, name="UTKface_Age")

In [ ]:
now_path = os.getcwd()
os.chdir(r"..\..\face-api\jupyter")
import __init__
from faceapi import FaceAPIManager
face_api_manager = FaceAPIManager(
    detector_model_path=r'C:\Users\jeffg\face-project\face-api\model_weights\face-yolov8-m.pt',
    encoder_model_path=r'C:\Users\jeffg\face-project\face-api\model_weights\feifei_face.pt',
    emotion_model_path=r"C:\Users\jeffg\face-project\face-api\model_weights\face-emotion.pth",
    
    age_proto_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\age_detector\age_deploy.prototxt', 
    age_model_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\age_detector\age_net.caffemodel', 
    gender_proto_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\gender_detector\gender_deploy.prototxt', 
    gender_model_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\gender_detector\gender_net.caffemodel', 

    device='cuda'
)
os.chdir(now_path)

In [ ]:
import cv2
from PIL import Image

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
cap.isOpened()

In [ ]:

while True:
    d, frame = cap.read()
    display_frame = frame.copy()
    faces = face_api_manager.handle(frame)
    
    label = "None"
    if len(faces) > 0:
        face = faces[0]

        x1,y1,x2,y2  = face.xyxy
        face_img = frame[y1:y2,x1:x2,:]

        
        res = model.predict(face_img, verbose=False)
        probs = res[0].probs
        prob = probs.top1conf.cpu().numpy().item()
        label = "male" if prob >= 0.5 else "female" 
        label = f"{label}-{prob}"

        #label = f"%s-%s-%s, %s - %.2f"%(face.emotion,  face.gender, face.age, label, pred_prob)
    
        cv2.putText(display_frame, label, (x1, y1),
                    cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))
        cv2.rectangle(display_frame, ( x1,y1), (x2,y2), (255, 255, 255))
        

    cv2.imshow('Webcam', display_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
res = model.predict(face_img, verbose=False)
probs = res[0].probs
prob = probs.top1conf.cpu().numpy().item()
label = "male" if prob >= 0.5 else "female" 